In [6]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import pandas as pd
import time

In [16]:
url = "https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag="
df=pd.DataFrame(columns=['price','locali','superficie','bagni','piano'])

In [23]:
pg=1
i=1
while i <= 10000:

    time.sleep(1)
    response = requests.get(url+str(pg))
    html_soup = bs(response.text, 'html.parser')
    item_containers = html_soup.find_all('div', class_ = 'listing-item_body--content')
    for item in item_containers:
        listing = []
        listing.append(item.a['href'])
            
        for feature in item.ul.find_all('li', class_ = 'lif__item'):
            try:
                if 'lif__pricing' in feature['class']:
                    listing.append(feature.text.strip().split(' ')[1])
                    next
                else:
                    if feature.find('div', class_='lif__text').text == 'locali':
                        listing.append(feature.span.text)
                    if feature.find('div', class_='lif__text').text == 'superficie':
                        listing.append(feature.span.text)
                    if feature.find('div', class_='lif__text').text == 'bagni':
                        listing.append(feature.span.text)
                    if feature.find('div', class_='lif__text').text == 'piano':
                        listing.append(feature.abbr.text.strip())
            except:
                next
        if len(listing) == 6:
            df=df.append(pd.Series(listing, index=['url','price','locali','superficie','bagni','piano']),ignore_index=True)
            i +=1
    pg+=1


In [28]:
json=df.to_json('announcements.json',orient='index')

In [32]:
test=pd.read_json('announcements.json',orient='index')